In [97]:
import pandas as pd
import re
import sqldf
from tqdm import tqdm, trange

In [26]:
# ri=pd.read_csv("F:/V2_RISK_INCIDENTS.csv")
# ri[ri['incident_date']>20091231].to_csv('rige2010.csv', index=False)
ri=pd.read_csv("rige2010.csv")
bw=pd.read_csv("bw0801.csv")

In [125]:
def strYear2Num(s):
    if s=="" or s=="nan":
        return s
    return s.split("/")[-1][:4]+s.split("/")[1].rjust(2,"0")+s.split("/")[0].rjust(2,"0")

cols=[
"reprisk_id",
"incident_date",
"severity",
"reach",
"novelty",
"environment",
"social",
"governance",
"cross_CUTTING"
]

ririd=ri.reprisk_id.unique()

# process bw date to int:

In [80]:
bw.AssessmentDate=bw.AssessmentDate.astype(str).apply(strYear2Num).astype(int)

In [86]:
bw.sort_values('repriskid').head(1)

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,cName,CNRR,repriskid,RRCleaned,Rrsector,doubt,handcoded,round
765,32243,364,classic fashion apparel industry ltd,Jordan,20190213,2019,classic fashion apparel industry ltd,Classic Fashion Apparel Industry Ltd (CFAI),2747.0,classic fashion apparel industry ltd,Personal and Household Goods,0.0,1,1.0


# half repriskID in BW not in RI

In [132]:
len(set(bw.repriskid.unique())-set(ririd)),bw.repriskid.nunique()

(221, 474)

# loop to calc:

In [110]:
riVars=[
"RRincidents",
"IC1_date",
"IC1_sev",
"IC1_reach",
"IC1_nov",
"IC1_env",
"IC1_soc",
"IC1_gov",
"IC1_cros"
]
bw[riVars]=''
bw[[x.replace("IC1","IC2") for x in riVars]]=''

In [158]:
for i,j in tqdm(bw.iterrows()):
    noMatch=False
    only1=False
    bwRID=j.repriskid
    if  pd.isna(bwRID) or int(bwRID) not in ririd:
        continue
    dateUB=j.AssessmentDate
    riTargRow=ri[(ri.reprisk_id==bwRID) & (ri.incident_date<=dateUB)]
    bw.loc[i,"RRincidents"]=riTargRow.incident_date.nunique() # each incident_date as an incident count
    try:
        earliestRIRow=riTargRow.sort_values('incident_date').iloc[0]
    except:
        noMatch=True

    try:
        secondEarliestRIRow=riTargRow.sort_values('incident_date').iloc[1]
    except:
        only1=True

    for riOrigColName, bwNewVarName in zip(cols[1:],riVars[1:]):
        if noMatch==False:
            bw.loc[i,bwNewVarName]=earliestRIRow[riOrigColName]
        if only1==False:
            bw.loc[i,bwNewVarName.replace("IC1","IC2")]=secondEarliestRIRow[riOrigColName]

# bw.loc[i,"IC1_date"]=earliestRIRow.incident_date
# bw.loc[i,"IC1_sev"]=earliestRIRow.incident_date
# bw.loc[i,"IC1_reach"]=earliestRIRow.incident_date
# bw.loc[i,"IC1_nov"]=earliestRIRow.incident_date
# bw.loc[i,"IC1_env"]=earliestRIRow.incident_date
# bw.loc[i,"IC1_soc"]=earliestRIRow.incident_date
# bw.loc[i,"IC1_gov"]=earliestRIRow.incident_date
# bw.loc[i,"IC1_cros"]=earliestRIRow.incident_date
     

6500it [00:06, 1013.98it/s]


In [115]:
# ri[(ri.reprisk_id==10) & (ri.incident_date<=20120101)].sort_values('incident_date')
ri[ri.reprisk_id==106981]

,reprisk_id,story_id,incident_date,unsharp_incident,related_countries,related_countries_codes,severity,reach,novelty,environment,...,ungc_principle_1,ungc_principle_2,ungc_principle_3,ungc_principle_4,ungc_principle_5,ungc_principle_6,ungc_principle_7,ungc_principle_8,ungc_principle_9,ungc_principle_10
35330,106981,43577,20130320,0,Jordan,JO,2,2,1,F,...,T,T,F,T,F,T,F,F,F,F


---
# 3 yrs window:

In [164]:
bw.AssessmentDate.min()
# cant be before 2010, so can still use `rige2010.csv`

20150106

In [173]:
bwAlt=bw.copy()

In [174]:
for i,j in tqdm(bw.iterrows()):
    noMatch,only1,only2=False,False,False
    bwRID=j.repriskid
    if  pd.isna(bwRID) or int(bwRID) not in ririd:
        continue
    dateUB=j.AssessmentDate
    dateLB=j.AssessmentDate-30000 #3 yrs window
    riTargRow=ri[(ri.reprisk_id==bwRID) & (ri.incident_date.isin(range(dateLB,dateUB+1)))] # +1 for inclusive
    bw.loc[i,"RRincidents"]=riTargRow.incident_date.nunique() # each incident_date as an incident count
    try:
        earliestRIRow=riTargRow.sort_values('incident_date').iloc[0]
    except:
        noMatch=True
    try:
        secondEarliestRIRow=riTargRow.sort_values('incident_date').iloc[1]
    except:
        only1=True
    try:
        secondEarliestRIRow=riTargRow.sort_values('incident_date').iloc[2]
    except:
        only2=True

    for riOrigColName, bwNewVarName in zip(cols[1:],riVars[1:]):
        if noMatch==False:
            bw.loc[i,bwNewVarName]=earliestRIRow[riOrigColName]
        if only1==False:
            bw.loc[i,bwNewVarName.replace("IC1","IC2")]=secondEarliestRIRow[riOrigColName]
        if only2==False:
            bw.loc[i,bwNewVarName.replace("IC1","IC3")]=secondEarliestRIRow[riOrigColName]

6500it [00:30, 214.05it/s]


---

In [172]:
bw.IC2_env.unique()

array(['', 'F', 'T'], dtype=object)

In [50]:
rit=ri[cols]
rit['idx']=rit.index
rit.head()

C:\Users\JselZ\AppData\Local\Temp\ipykernel_32764\283144027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rit['idx']=rit.index


,reprisk_id,incident_date,severity,reach,novelty,environment,social,governance,cross_CUTTING,idx
0,10,20100324,1,1,2,T,T,F,T,0
1,10,20100413,2,2,1,F,T,F,T,1
2,10,20100518,2,1,1,F,F,F,T,2
3,10,20101014,2,1,2,F,T,F,T,3
4,10,20101026,1,2,2,T,F,F,T,4


# match assessmentDate

In [61]:
result = pd.merge(rit, bw, left_on='reprisk_id', right_on='repriskid', how='left')
rit['assessDate']=result['AssessmentDate']

C:\Users\JselZ\AppData\Local\Temp\ipykernel_32764\1108073747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rit['assessDate']=result['AssessmentDate']


In [63]:
rit['assessDate'].value_counts(dropna=0)

NaN           875352
15/09/2019       120
16/09/2018       110
17/01/2021       110
10/9/2017        100
               ...  
11/2/2015          1
7/11/2019          1
23/07/2018         1
24/03/2019         1
7/9/2021           1
Name: assessDate, Length: 697, dtype: int64

# process `rit.assessDate`:

In [44]:
ri.sort_values('incident_date').groupby('reprisk_id').first()[cols]
# ri.groupby('reprisk_id')['incident_date'].nsmallest(2)

,story_id,incident_date,unsharp_incident,related_countries,related_countries_codes,severity,reach,novelty,environment,social,...,ungc_principle_1,ungc_principle_2,ungc_principle_3,ungc_principle_4,ungc_principle_5,ungc_principle_6,ungc_principle_7,ungc_principle_8,ungc_principle_9,ungc_principle_10
reprisk_id,,,,,,,,,,,,,,,,,,,,,
2,12923,20100108,0,United States of America,US,2,2,2,F,T,...,F,F,F,F,F,F,F,F,F,F
10,14020,20100324,0,China,CN,1,1,2,T,T,...,F,F,F,F,F,F,T,T,T,F
11,13063,20100111,1,Chile,CL,2,2,1,T,T,...,F,F,F,F,F,F,T,T,T,F
12,14186,20100322,0,Brazil,BR,1,1,2,F,T,...,F,F,F,F,F,F,F,F,F,F
13,14140,20100405,0,Bahrain;United Kingdom of Great Britain and No...,BH;GB;US,2,3,2,F,F,...,F,F,F,F,F,F,F,F,F,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677636,8431854,20231016,0,Iran (Islamic Republic of);Unspecified,IR;,1,2,2,F,F,...,F,F,F,F,F,F,F,F,F,F
2677696,8431854,20231016,0,Iran (Islamic Republic of);Unspecified,IR;,1,2,2,F,F,...,F,F,F,F,F,F,F,F,F,F
2677729,8431854,20231016,0,Iran (Islamic Republic of);Unspecified,IR;,1,2,2,F,F,...,F,F,F,F,F,F,F,F,F,F


In [18]:
for i in ri.columns:
    print(i)

reprisk_id
story_id
incident_date
unsharp_incident
related_countries
related_countries_codes
severity
reach
novelty
environment
social
governance
cross_CUTTING
animal_mistreatment
anti_competititve_practices
child_labor
climate_ghg_pollution
controversial_prod_services
money_related
discrimination_in_employment
executive_compensation_issues
forced_labor
fraud
freedom_of_association
human_rights_abuses
impacts_on_communities
impacts_on_landscapes
local_participation_issues
local_pollution
misleading_communication
occupational_health
other_esg_issues
overuse_and_wasting
poor_employment_conditions
prod_health_environ_issues
social_discrimination
supply_chain_issues
tax_evasion
tax_optimization
violation_of_intl_standards
violation_of_natl_legislation
waste_issues
abusive_illegal_fishing
access_to_prod_services
agri_commodity_speculation
airborne_pollutants
alcohol
animal_transportation
arctic_drilling
asbestos
automatic_semi_auto_weapons
biological_weapons
chemical_weapons
cluster_munitio

---
# export

In [161]:
bw.to_csv('bw0802.csv',index=False)

In [177]:
bw.to_csv('bw0803.csv',index=False)